In [1]:
import os
import sys
import time
import matplotlib
import numpy as np
import pandas as pd
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score, f1_score, recall_score
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.layers import Input, Conv1D, MaxPooling1D, Flatten, Dense, Dropout, Softmax, Multiply, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential
# os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
all_start_time = time.time()

2023-04-29 20:15:10.733504: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0


In [2]:
print(pd.__version__)
print(np.__version__)
print(matplotlib.__version__)
print(sklearn.__version__)
print(tf.__version__)
print(keras.__version__)
!nvidia-smi

1.3.4
1.19.5
3.4.3
1.0.2
2.5.0
2.5.0
Sat Apr 29 20:15:12 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            On   | 00000000:00:07.0 Off |                  Off |
| N/A   29C    P8    11W /  70W |      0MiB / 16384MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                   

In [3]:
cols="""duration,
protocol_type,
service,
flag,
src_bytes,
dst_bytes,
land,
wrong_fragment,
urgent,
hot,
num_failed_logins,
logged_in,
num_compromised,
root_shell,
su_attempted,
num_root,
num_file_creations,
num_shells,
num_access_files,
num_outbound_cmds,
is_host_login,
is_guest_login,
count,
srv_count,
serror_rate,
srv_serror_rate,
rerror_rate,
srv_rerror_rate,
same_srv_rate,
diff_srv_rate,
srv_diff_host_rate,
dst_host_count,
dst_host_srv_count,
dst_host_same_srv_rate,
dst_host_diff_srv_rate,
dst_host_same_src_port_rate,
dst_host_srv_diff_host_rate,
dst_host_serror_rate,
dst_host_srv_serror_rate,
dst_host_rerror_rate,
dst_host_srv_rerror_rate"""

columns=[]
for c in cols.split(','):
    if(c.strip()):
       columns.append(c.strip())

columns.append('target')
#print(columns)
print(len(columns))

42


In [4]:
attacks_types = {
    'normal': 'normal',
'back': 'dos',
'buffer_overflow': 'u2r',
'ftp_write': 'r2l',
'guess_passwd': 'r2l',
'imap': 'r2l',
'ipsweep': 'probe',
'land': 'dos',
'loadmodule': 'u2r',
'multihop': 'r2l',
'neptune': 'dos',
'nmap': 'probe',
'perl': 'u2r',
'phf': 'r2l',
'pod': 'dos',
'portsweep': 'probe',
'rootkit': 'u2r',
'satan': 'probe',
'smurf': 'dos',
'spy': 'r2l',
'teardrop': 'dos',
'warezclient': 'r2l',
'warezmaster': 'r2l',
}

In [5]:
path1 = "../../../kddcup.data"
path2 = "../../../kddcup.data_10_percent"
df = pd.read_csv(path1,names=columns)

#Adding Attack Type column
df['Attack Type'] = df.target.apply(lambda r:attacks_types[r[:-1]])

df.head()

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,target,Attack Type
0,0,tcp,http,SF,215,45076,0,0,0,0,...,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,normal.,normal
1,0,tcp,http,SF,162,4528,0,0,0,0,...,1.0,0.0,1.00,0.0,0.0,0.0,0.0,0.0,normal.,normal
2,0,tcp,http,SF,236,1228,0,0,0,0,...,1.0,0.0,0.50,0.0,0.0,0.0,0.0,0.0,normal.,normal
3,0,tcp,http,SF,233,2032,0,0,0,0,...,1.0,0.0,0.33,0.0,0.0,0.0,0.0,0.0,normal.,normal
4,0,tcp,http,SF,239,486,0,0,0,0,...,1.0,0.0,0.25,0.0,0.0,0.0,0.0,0.0,normal.,normal


In [6]:
df = df.dropna('columns')# drop columns with NaN
df = df[[col for col in df if df[col].nunique() > 1]]# keep columns where there are more than 1 unique values

df.drop('num_root',axis = 1,inplace = True)
df.drop('srv_serror_rate',axis = 1,inplace = True)
df.drop('srv_rerror_rate',axis = 1, inplace=True)
df.drop('dst_host_srv_serror_rate',axis = 1, inplace=True)
df.drop('dst_host_serror_rate',axis = 1, inplace=True)
df.drop('dst_host_rerror_rate',axis = 1, inplace=True)
df.drop('dst_host_srv_rerror_rate',axis = 1, inplace=True)
df.drop('dst_host_same_srv_rate',axis = 1, inplace=True)

df.drop('is_host_login',axis = 1, inplace=True)

#protocol_type feature mapping
pmap = {'icmp':0,'tcp':1,'udp':2}
df['protocol_type'] = df['protocol_type'].map(pmap)

#flag feature mapping
fmap = {'SF':0,'S0':1,'REJ':2,'RSTR':3,'RSTO':4,'SH':5 ,'S1':6 ,'S2':7,'RSTOS0':8,'S3':9 ,'OTH':10}
df['flag'] = df['flag'].map(fmap)

#attack type feature mapping
amap = {'dos':0,'normal':1,'probe':2,'r2l':3,'u2r':4}
df['Attack Type'] = df['Attack Type'].map(amap)

df.drop('service',axis = 1,inplace= True)

df = df.drop(['target',], axis=1)
print(df.shape)

/tmp/ipykernel_48085/1196107924.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.dropna will be keyword-only
  df = df.dropna('columns')# drop columns with NaN


(4898431, 31)


In [7]:
# Target variable and train set
Y = df[['Attack Type']]
X = df.drop(['Attack Type',], axis=1)
batch_size=64

sc = MinMaxScaler()
X = sc.fit_transform(X)

# Split test and train data 
# X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.33, shuffle=False)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.33, random_state=42, shuffle=True)
print(X_train.shape, X_test.shape)
print(Y_train.shape, Y_test.shape)

(3281948, 30) (1616483, 30)
(3281948, 1) (1616483, 1)


## Attention CNN

### Attention_CNN2(bn)

In [8]:
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Dense, Dropout, Input, BatchNormalization, GlobalMaxPooling1D

def attention_layer(inputs):
    attention_weights = Dense(units=128, activation='softmax')(inputs)
    attention_weights = Flatten()(attention_weights)
    attention_weights = Multiply()([inputs, attention_weights])
    return attention_weights

input_layer = Input(shape=(30, 1))
conv_layer1 = Conv1D(64, 3, padding="same", activation="relu")(input_layer)
pool_layer1 = MaxPooling1D(pool_size=(2))(conv_layer1)
conv_layer2 = Conv1D(128, 3, padding="same", activation="relu")(pool_layer1)
pool_layer2 = MaxPooling1D(pool_size=(2))(conv_layer2)
flatten_layer = Flatten()(pool_layer2)
dense_layer1 = Dense(128, activation="relu")(flatten_layer)
bn_layer1 = BatchNormalization()(dense_layer1)
attention_layer1 = attention_layer(bn_layer1)
dense_layer2 = Dense(64, activation="relu")(attention_layer1)
bn_layer2 = BatchNormalization()(dense_layer2)
output_layer = Dense(5, activation="softmax")(bn_layer2)
attention_cnn2bn_model = Model(inputs=input_layer, outputs=output_layer)

attention_cnn2bn_model.compile(loss ='sparse_categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
print(attention_cnn2bn_model.summary())
start_time = time.time()
attention_cnn2bn_model.fit(X_train.reshape((-1,30,1)), Y_train.values.ravel(), epochs=10, batch_size=batch_size)
end_time = time.time()
attention_cnn2bn_time = end_time-start_time
print("Training time: ", attention_cnn2bn_time)

2023-04-29 20:15:53.521104: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2023-04-29 20:15:53.543072: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-04-29 20:15:53.544210: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:00:07.0 name: Tesla T4 computeCapability: 7.5
coreClock: 1.59GHz coreCount: 40 deviceMemorySize: 15.74GiB deviceMemoryBandwidth: 298.08GiB/s
2023-04-29 20:15:53.544271: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
2023-04-29 20:15:53.547645: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublas.so.11
2023-04-29 20:15:53.547744: I tensorflow/stream_executor/platform/default/d

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 30, 1)]      0                                            
__________________________________________________________________________________________________
conv1d (Conv1D)                 (None, 30, 64)       256         input_1[0][0]                    
__________________________________________________________________________________________________
max_pooling1d (MaxPooling1D)    (None, 15, 64)       0           conv1d[0][0]                     
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, 15, 128)      24704       max_pooling1d[0][0]              
______________________________________________________________________________________________

2023-04-29 20:15:54.994284: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)
2023-04-29 20:15:54.994746: I tensorflow/core/platform/profile_utils/cpu_utils.cc:114] CPU Frequency: 2999760000 Hz


Epoch 1/10


2023-04-29 20:15:55.750816: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudnn.so.8
2023-04-29 20:15:56.285141: I tensorflow/stream_executor/cuda/cuda_dnn.cc:359] Loaded cuDNN version 8101
2023-04-29 20:15:56.918067: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublas.so.11
2023-04-29 20:15:57.340078: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublasLt.so.11


51281/51281 [==============================] - 152s 3ms/step - loss: 0.0045 - accuracy: 0.9990
Epoch 2/10
51281/51281 [==============================] - 150s 3ms/step - loss: 0.0019 - accuracy: 0.9995
Epoch 3/10
51281/51281 [==============================] - 152s 3ms/step - loss: 0.0017 - accuracy: 0.9996
Epoch 4/10
51281/51281 [==============================] - 151s 3ms/step - loss: 0.0015 - accuracy: 0.9996
Epoch 5/10
51281/51281 [==============================] - 151s 3ms/step - loss: 0.0015 - accuracy: 0.9996
Epoch 6/10
51281/51281 [==============================] - 150s 3ms/step - loss: 0.0014 - accuracy: 0.9996
Epoch 7/10
51281/51281 [==============================] - 149s 3ms/step - loss: 0.0014 - accuracy: 0.9996
Epoch 8/10
51281/51281 [==============================] - 151s 3ms/step - loss: 0.0014 - accuracy: 0.9997
Epoch 9/10
51281/51281 [==============================] - 150s 3ms/step - loss: 0.0013 - accuracy: 0.9997
Epoch 10/10
51281/51281 [==============================] 

In [9]:
attention_cnn2bn_preds_train = attention_cnn2bn_model.predict(X_train.reshape((-1,30,1)))
attention_cnn2bn_test = attention_cnn2bn_model.predict(X_test.reshape((-1,30,1)))
print("attention_cnn2bn_model")
attention_cnn2bn_train_acc = accuracy_score(Y_train, np.argmax(attention_cnn2bn_preds_train, axis=1))
attention_cnn2bn_test_acc = accuracy_score(Y_test, np.argmax(attention_cnn2bn_test, axis=1))
attention_cnn2bn_train_f1 = f1_score(Y_train, np.argmax(attention_cnn2bn_preds_train, axis=1), average='macro')
attention_cnn2bn_test_f1 = f1_score(Y_test, np.argmax(attention_cnn2bn_test, axis=1), average='macro')
attention_cnn2bn_train_recall = recall_score(Y_train, np.argmax(attention_cnn2bn_preds_train, axis=1), average='macro')
attention_cnn2bn_test_recall = recall_score(Y_test, np.argmax(attention_cnn2bn_test, axis=1), average='macro')
print("Training Accuracy:", attention_cnn2bn_train_acc)
print("Testing Accuracy:", attention_cnn2bn_test_acc)
print("Training f1:", attention_cnn2bn_train_f1)
print("Testing f1:", attention_cnn2bn_test_f1)
print("Training recall:", attention_cnn2bn_train_recall)
print("Testing recall:", attention_cnn2bn_test_recall)

attention_cnn2bn_model
Training Accuracy: 0.9996876854843526
Testing Accuracy: 0.999612739509169
Training f1: 0.852972854380087
Testing f1: 0.8286464584400639
Training recall: 0.8290413474024495
Testing recall: 0.8068572856283784


## 改进

### Attention_CNN3(dropout1==0.1)

In [10]:
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Dense, Dropout, Input, BatchNormalization, GlobalMaxPooling1D

def attention_layer(inputs):
    attention_weights = Dense(units=128, activation='softmax')(inputs)
    attention_weights = Flatten()(attention_weights)
    attention_weights = Multiply()([inputs, attention_weights])
    return attention_weights

input_layer = Input(shape=(30, 1))
conv_layer1 = Conv1D(64, 3, padding="same", activation="relu")(input_layer)
pool_layer1 = MaxPooling1D(pool_size=(2))(conv_layer1)
conv_layer2 = Conv1D(128, 3, padding="same", activation="relu")(pool_layer1)
pool_layer2 = MaxPooling1D(pool_size=(2))(conv_layer2)
flatten_layer = Flatten()(pool_layer2)
dense_layer1 = Dense(128, activation="relu")(flatten_layer)
bn_layer1 = BatchNormalization()(dense_layer1)
dropout_layer1 = Dropout(0.01)(bn_layer1)
attention_layer1 = attention_layer(dropout_layer1)
dense_layer2 = Dense(64, activation="relu")(attention_layer1)
dropout_layer2 = Dropout(0.01)(dense_layer2)
output_layer = Dense(5, activation="softmax")(dropout_layer2)
attention_cnn3dropout1_model = Model(inputs=input_layer, outputs=output_layer)

attention_cnn3dropout1_model.compile(loss ='sparse_categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
print(attention_cnn3dropout1_model.summary())
start_time = time.time()
attention_cnn3dropout1_model.fit(X_train.reshape((-1,30,1)), Y_train.values.ravel(), epochs=10, batch_size=batch_size)
end_time = time.time()
attention_cnn3dropout1_time = end_time-start_time
print("Training time: ", attention_cnn3dropout1_time)

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 30, 1)]      0                                            
__________________________________________________________________________________________________
conv1d_2 (Conv1D)               (None, 30, 64)       256         input_2[0][0]                    
__________________________________________________________________________________________________
max_pooling1d_2 (MaxPooling1D)  (None, 15, 64)       0           conv1d_2[0][0]                   
__________________________________________________________________________________________________
conv1d_3 (Conv1D)               (None, 15, 128)      24704       max_pooling1d_2[0][0]            
____________________________________________________________________________________________

In [11]:
cnn3dropout1_preds_train = attention_cnn3dropout1_model.predict(X_train.reshape((-1,30,1)))
cnn3dropout1_test = attention_cnn3dropout1_model.predict(X_test.reshape((-1,30,1)))
print("attention_cnn3dropout1_model")
attention_cnn3dropout1_train_acc = accuracy_score(Y_train, np.argmax(cnn3dropout1_preds_train, axis=1))
attention_cnn3dropout1_test_acc = accuracy_score(Y_test, np.argmax(cnn3dropout1_test, axis=1))
attention_cnn3dropout1_train_f1 = f1_score(Y_train, np.argmax(cnn3dropout1_preds_train, axis=1), average='macro')
attention_cnn3dropout1_test_f1 = f1_score(Y_test, np.argmax(cnn3dropout1_test, axis=1), average='macro')
attention_cnn3dropout1_train_recall = recall_score(Y_train, np.argmax(cnn3dropout1_preds_train, axis=1), average='macro')
attention_cnn3dropout1_test_recall = recall_score(Y_test, np.argmax(cnn3dropout1_test, axis=1), average='macro')
print("Training Accuracy:", attention_cnn3dropout1_train_acc)
print("Testing Accuracy:", attention_cnn3dropout1_test_acc)
print("Training f1:", attention_cnn3dropout1_train_f1)
print("Testing f1:", attention_cnn3dropout1_test_f1)
print("Training recall:", attention_cnn3dropout1_train_recall)
print("Testing recall:", attention_cnn3dropout1_test_recall)

attention_cnn3dropout1_model
Training Accuracy: 0.999694998214475
Testing Accuracy: 0.9996207816599371
Training f1: 0.7546841142258833
Testing f1: 0.7494464146120455
Training recall: 0.7474162876668743
Testing recall: 0.7472531478720398


### Attention_CNN3(dropout2==0.01)

In [12]:
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Dense, Dropout, Input, BatchNormalization, GlobalMaxPooling1D

def attention_layer(inputs):
    attention_weights = Dense(units=128, activation='softmax')(inputs)
    attention_weights = Flatten()(attention_weights)
    attention_weights = Multiply()([inputs, attention_weights])
    return attention_weights

input_layer = Input(shape=(30, 1))
conv_layer1 = Conv1D(64, 3, padding="same", activation="relu")(input_layer)
pool_layer1 = MaxPooling1D(pool_size=(2))(conv_layer1)
conv_layer2 = Conv1D(128, 3, padding="same", activation="relu")(pool_layer1)
pool_layer2 = MaxPooling1D(pool_size=(2))(conv_layer2)
flatten_layer = Flatten()(pool_layer2)
dense_layer1 = Dense(128, activation="relu")(flatten_layer)
bn_layer1 = BatchNormalization()(dense_layer1)
dropout_layer1 = Dropout(0.01)(bn_layer1)
attention_layer1 = attention_layer(dropout_layer1)
dense_layer2 = Dense(64, activation="relu")(attention_layer1)
dropout_layer2 = Dropout(0.01)(dense_layer2)
output_layer = Dense(5, activation="softmax")(dropout_layer2)
attention_cnn3dropout2_model = Model(inputs=input_layer, outputs=output_layer)

attention_cnn3dropout2_model.compile(loss ='sparse_categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
print(attention_cnn3dropout2_model.summary())
start_time = time.time()
attention_cnn3dropout2_model.fit(X_train.reshape((-1,30,1)), Y_train.values.ravel(), epochs=10, batch_size=batch_size)
end_time = time.time()
attention_cnn3dropout2_time = end_time-start_time
print("Training time: ", attention_cnn3dropout2_time)

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 30, 1)]      0                                            
__________________________________________________________________________________________________
conv1d_4 (Conv1D)               (None, 30, 64)       256         input_3[0][0]                    
__________________________________________________________________________________________________
max_pooling1d_4 (MaxPooling1D)  (None, 15, 64)       0           conv1d_4[0][0]                   
__________________________________________________________________________________________________
conv1d_5 (Conv1D)               (None, 15, 128)      24704       max_pooling1d_4[0][0]            
____________________________________________________________________________________________

In [13]:
cnn3dropout2_preds_train = attention_cnn3dropout2_model.predict(X_train.reshape((-1,30,1)))
cnn3dropout2_test = attention_cnn3dropout2_model.predict(X_test.reshape((-1,30,1)))
print("attention_cnn3dropout2_model")
attention_cnn3dropout2_train_acc = accuracy_score(Y_train, np.argmax(cnn3dropout2_preds_train, axis=1))
attention_cnn3dropout2_test_acc = accuracy_score(Y_test, np.argmax(cnn3dropout2_test, axis=1))
attention_cnn3dropout2_train_f1 = f1_score(Y_train, np.argmax(cnn3dropout2_preds_train, axis=1), average='macro')
attention_cnn3dropout2_test_f1 = f1_score(Y_test, np.argmax(cnn3dropout2_test, axis=1), average='macro')
attention_cnn3dropout2_train_recall = recall_score(Y_train, np.argmax(cnn3dropout2_preds_train, axis=1), average='macro')
attention_cnn3dropout2_test_recall = recall_score(Y_test, np.argmax(cnn3dropout2_test, axis=1), average='macro')
print("Training Accuracy:", attention_cnn3dropout2_train_acc)
print("Testing Accuracy:", attention_cnn3dropout2_test_acc)
print("Training f1:", attention_cnn3dropout2_train_f1)
print("Testing f1:", attention_cnn3dropout2_test_f1)
print("Training recall:", attention_cnn3dropout2_train_recall)
print("Testing recall:", attention_cnn3dropout2_test_recall)

attention_cnn3dropout2_model
Training Accuracy: 0.9996291836433728
Testing Accuracy: 0.9995756218902395
Training f1: 0.7483151476064495
Testing f1: 0.7463092866496739
Training recall: 0.7465699444361118
Testing recall: 0.7519383077170098


In [14]:
train_acc = {"attention_cnn2bn_train_acc":attention_cnn2bn_train_acc, 
            "attention_cnn3dropout1_train_acc":attention_cnn3dropout1_train_acc, "attention_cnn3dropout2_train_acc":attention_cnn3dropout2_train_acc}
test_acc = {"attention_cnn2bn_test_acc":attention_cnn2bn_test_acc,
            "attention_cnn3dropout1_test_acc":attention_cnn3dropout1_test_acc, "attention_cnn3dropout2_test_acc":attention_cnn3dropout2_test_acc}

In [15]:
train_f1 = {"attention_cnn2bn_train_f1":attention_cnn2bn_train_f1, 
            "attention_cnn3dropout1_train_f1":attention_cnn3dropout1_train_f1, "attention_cnn3dropout2_train_f1":attention_cnn3dropout2_train_f1}
test_f1 = {"attention_cnn2bn_test_f1":attention_cnn2bn_test_f1, 
            "attention_cnn3dropout1_test_f1":attention_cnn3dropout1_test_f1, "attention_cnn3dropout2_test_f1":attention_cnn3dropout2_test_f1}

In [16]:
train_recall = {"attention_cnn2bn_train_recall":attention_cnn2bn_train_recall, 
            "attention_cnn3dropout1_train_recall":attention_cnn3dropout1_train_recall, "attention_cnn3dropout2_train_recall":attention_cnn3dropout2_train_recall}
test_recall = {"attention_cnn2bn_test_recall":attention_cnn2bn_test_recall, 
            "attention_cnn3dropout1_test_recall":attention_cnn3dropout1_test_recall, "attention_cnn3dropout2_test_recall":attention_cnn3dropout2_test_recall}

In [17]:
print(train_acc, test_acc, train_f1, test_f1, train_recall, test_recall, sep='\n')

{'attention_cnn2bn_train_acc': 0.9996876854843526, 'attention_cnn3dropout1_train_acc': 0.999694998214475, 'attention_cnn3dropout2_train_acc': 0.9996291836433728}
{'attention_cnn2bn_test_acc': 0.999612739509169, 'attention_cnn3dropout1_test_acc': 0.9996207816599371, 'attention_cnn3dropout2_test_acc': 0.9995756218902395}
{'attention_cnn2bn_train_f1': 0.852972854380087, 'attention_cnn3dropout1_train_f1': 0.7546841142258833, 'attention_cnn3dropout2_train_f1': 0.7483151476064495}
{'attention_cnn2bn_test_f1': 0.8286464584400639, 'attention_cnn3dropout1_test_f1': 0.7494464146120455, 'attention_cnn3dropout2_test_f1': 0.7463092866496739}
{'attention_cnn2bn_train_recall': 0.8290413474024495, 'attention_cnn3dropout1_train_recall': 0.7474162876668743, 'attention_cnn3dropout2_train_recall': 0.7465699444361118}
{'attention_cnn2bn_test_recall': 0.8068572856283784, 'attention_cnn3dropout1_test_recall': 0.7472531478720398, 'attention_cnn3dropout2_test_recall': 0.7519383077170098}


In [18]:
def dic_to_txt(dic, filepath):
    # 遍历字典的元素，将每项元素的key和value分拆组成字符串，注意添加分隔符和换行符
    file = open(filepath, 'w')
    for k, v in dic.items():
        file.write(str(k) + ' ' + str(v) + '\n')

    # 注意关闭文件
    file.close()
    print('done')

In [19]:
dic_to_txt(train_acc, './AttentionCNN_reduce_train_acc.txt')
dic_to_txt(test_acc, './AttentionCNN_reduce_test_acc.txt')

done
done


In [20]:
dic_to_txt(train_f1, './AttentionCNN_reduce_train_f1.txt')
dic_to_txt(test_f1, './AttentionCNN_reduce_test_f1.txt')

done
done


In [21]:
dic_to_txt(train_recall, './AttentionCNN_reduce_train_recall.txt')
dic_to_txt(test_recall, './AttentionCNN_reduce_test_recall.txt')

done
done


--------------------------------------------------------------------------------------

--------------------------------------------------------------------------------------

--------------------------------------------------------------------------------------

In [22]:
import os
import sys
import time
import matplotlib
import numpy as np
import pandas as pd
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score, f1_score, recall_score
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.layers import Input, Conv1D, MaxPooling1D, Flatten, Dense, Dropout, Softmax, Multiply, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential
# os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

In [23]:
print(pd.__version__)
print(np.__version__)
print(matplotlib.__version__)
print(sklearn.__version__)
print(tf.__version__)
print(keras.__version__)
!nvidia-smi

1.3.4
1.19.5
3.4.3
1.0.2
2.5.0
2.5.0
Sat Apr 29 21:33:09 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            On   | 00000000:00:07.0 Off |                  Off |
| N/A   40C    P0    29W /  70W |  15641MiB / 16384MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                   

In [24]:
cols="""duration,
protocol_type,
service,
flag,
src_bytes,
dst_bytes,
land,
wrong_fragment,
urgent,
hot,
num_failed_logins,
logged_in,
num_compromised,
root_shell,
su_attempted,
num_root,
num_file_creations,
num_shells,
num_access_files,
num_outbound_cmds,
is_host_login,
is_guest_login,
count,
srv_count,
serror_rate,
srv_serror_rate,
rerror_rate,
srv_rerror_rate,
same_srv_rate,
diff_srv_rate,
srv_diff_host_rate,
dst_host_count,
dst_host_srv_count,
dst_host_same_srv_rate,
dst_host_diff_srv_rate,
dst_host_same_src_port_rate,
dst_host_srv_diff_host_rate,
dst_host_serror_rate,
dst_host_srv_serror_rate,
dst_host_rerror_rate,
dst_host_srv_rerror_rate"""

columns=[]
for c in cols.split(','):
    if(c.strip()):
       columns.append(c.strip())

columns.append('target')
#print(columns)
print(len(columns))

42


In [25]:
attacks_types = {
    'normal': 'normal',
'back': 'dos',
'buffer_overflow': 'u2r',
'ftp_write': 'r2l',
'guess_passwd': 'r2l',
'imap': 'r2l',
'ipsweep': 'probe',
'land': 'dos',
'loadmodule': 'u2r',
'multihop': 'r2l',
'neptune': 'dos',
'nmap': 'probe',
'perl': 'u2r',
'phf': 'r2l',
'pod': 'dos',
'portsweep': 'probe',
'rootkit': 'u2r',
'satan': 'probe',
'smurf': 'dos',
'spy': 'r2l',
'teardrop': 'dos',
'warezclient': 'r2l',
'warezmaster': 'r2l',
}

In [26]:
path1 = "../../../kddcup.data"
path2 = "../../../kddcup.data_10_percent"
df = pd.read_csv(path1,names=columns)

#Adding Attack Type column
df['Attack Type'] = df.target.apply(lambda r:attacks_types[r[:-1]])

df.head()

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,target,Attack Type
0,0,tcp,http,SF,215,45076,0,0,0,0,...,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,normal.,normal
1,0,tcp,http,SF,162,4528,0,0,0,0,...,1.0,0.0,1.00,0.0,0.0,0.0,0.0,0.0,normal.,normal
2,0,tcp,http,SF,236,1228,0,0,0,0,...,1.0,0.0,0.50,0.0,0.0,0.0,0.0,0.0,normal.,normal
3,0,tcp,http,SF,233,2032,0,0,0,0,...,1.0,0.0,0.33,0.0,0.0,0.0,0.0,0.0,normal.,normal
4,0,tcp,http,SF,239,486,0,0,0,0,...,1.0,0.0,0.25,0.0,0.0,0.0,0.0,0.0,normal.,normal


In [27]:
df = df.dropna('columns')# drop columns with NaN
df = df[[col for col in df if df[col].nunique() > 1]]# keep columns where there are more than 1 unique values

# df.drop('num_root',axis = 1,inplace = True)
# df.drop('srv_serror_rate',axis = 1,inplace = True)
# df.drop('srv_rerror_rate',axis = 1, inplace=True)
# df.drop('dst_host_srv_serror_rate',axis = 1, inplace=True)
# df.drop('dst_host_serror_rate',axis = 1, inplace=True)
# df.drop('dst_host_rerror_rate',axis = 1, inplace=True)
# df.drop('dst_host_srv_rerror_rate',axis = 1, inplace=True)
# df.drop('dst_host_same_srv_rate',axis = 1, inplace=True)

# df.drop('is_host_login',axis = 1, inplace=True)

#protocol_type feature mapping
pmap = {'icmp':0,'tcp':1,'udp':2}
df['protocol_type'] = df['protocol_type'].map(pmap)

#flag feature mapping
fmap = {'SF':0,'S0':1,'REJ':2,'RSTR':3,'RSTO':4,'SH':5 ,'S1':6 ,'S2':7,'RSTOS0':8,'S3':9 ,'OTH':10}
df['flag'] = df['flag'].map(fmap)

#attack type feature mapping
amap = {'dos':0,'normal':1,'probe':2,'r2l':3,'u2r':4}
df['Attack Type'] = df['Attack Type'].map(amap)

df.drop('service',axis = 1,inplace= True)

df = df.drop(['target',], axis=1)
print(df.shape)

/tmp/ipykernel_48085/4268764986.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.dropna will be keyword-only
  df = df.dropna('columns')# drop columns with NaN


(4898431, 40)


In [28]:
# Target variable and train set
Y = df[['Attack Type']]
X = df.drop(['Attack Type',], axis=1)
batch_size=batch_size

sc = MinMaxScaler()
X = sc.fit_transform(X)

# Split test and train data 
# X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.33, shuffle=False)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.33, random_state=42, shuffle=True)
print(X_train.shape, X_test.shape)
print(Y_train.shape, Y_test.shape)

(3281948, 39) (1616483, 39)
(3281948, 1) (1616483, 1)


## Unprocessed Attention CNN

### Attention_CNN2(bn)

In [29]:
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Dense, Dropout, Input, BatchNormalization, GlobalMaxPooling1D

def attention_layer(inputs):
    attention_weights = Dense(units=128, activation='softmax')(inputs)
    attention_weights = Flatten()(attention_weights)
    attention_weights = Multiply()([inputs, attention_weights])
    return attention_weights

input_layer = Input(shape=(39, 1))
conv_layer1 = Conv1D(64, 3, padding="same", activation="relu")(input_layer)
pool_layer1 = MaxPooling1D(pool_size=(2))(conv_layer1)
conv_layer2 = Conv1D(128, 3, padding="same", activation="relu")(pool_layer1)
pool_layer2 = MaxPooling1D(pool_size=(2))(conv_layer2)
flatten_layer = Flatten()(pool_layer2)
dense_layer1 = Dense(128, activation="relu")(flatten_layer)
bn_layer1 = BatchNormalization()(dense_layer1)
attention_layer1 = attention_layer(bn_layer1)
dense_layer2 = Dense(64, activation="relu")(attention_layer1)
bn_layer2 = BatchNormalization()(dense_layer2)
output_layer = Dense(5, activation="softmax")(bn_layer2)
attention_cnn2bn_model = Model(inputs=input_layer, outputs=output_layer)

attention_cnn2bn_model.compile(loss ='sparse_categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
print(attention_cnn2bn_model.summary())
start_time = time.time()
attention_cnn2bn_model.fit(X_train.reshape((-1,39,1)), Y_train.values.ravel(), epochs=10, batch_size=batch_size)
end_time = time.time()
attention_cnn2bn_time = end_time-start_time
print("Training time: ", attention_cnn2bn_time)

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(None, 39, 1)]      0                                            
__________________________________________________________________________________________________
conv1d_6 (Conv1D)               (None, 39, 64)       256         input_4[0][0]                    
__________________________________________________________________________________________________
max_pooling1d_6 (MaxPooling1D)  (None, 19, 64)       0           conv1d_6[0][0]                   
__________________________________________________________________________________________________
conv1d_7 (Conv1D)               (None, 19, 128)      24704       max_pooling1d_6[0][0]            
____________________________________________________________________________________________

In [30]:
attention_cnn2bn_preds_train = attention_cnn2bn_model.predict(X_train.reshape((-1,39,1)))
attention_cnn2bn_test = attention_cnn2bn_model.predict(X_test.reshape((-1,39,1)))
print("attention_cnn2bn_model")
attention_cnn2bn_train_acc = accuracy_score(Y_train, np.argmax(attention_cnn2bn_preds_train, axis=1))
attention_cnn2bn_test_acc = accuracy_score(Y_test, np.argmax(attention_cnn2bn_test, axis=1))
attention_cnn2bn_train_f1 = f1_score(Y_train, np.argmax(attention_cnn2bn_preds_train, axis=1), average='macro')
attention_cnn2bn_test_f1 = f1_score(Y_test, np.argmax(attention_cnn2bn_test, axis=1), average='macro')
attention_cnn2bn_train_recall = recall_score(Y_train, np.argmax(attention_cnn2bn_preds_train, axis=1), average='macro')
attention_cnn2bn_test_recall = recall_score(Y_test, np.argmax(attention_cnn2bn_test, axis=1), average='macro')
print("Training Accuracy:", attention_cnn2bn_train_acc)
print("Testing Accuracy:", attention_cnn2bn_test_acc)
print("Training f1:", attention_cnn2bn_train_f1)
print("Testing f1:", attention_cnn2bn_test_f1)
print("Training recall:", attention_cnn2bn_train_recall)
print("Testing recall:", attention_cnn2bn_test_recall)

attention_cnn2bn_model
Training Accuracy: 0.9997178504961078
Testing Accuracy: 0.9996783139692778
Training f1: 0.7548793939128742
Testing f1: 0.7543989859493948
Training recall: 0.7485729355482988
Testing recall: 0.7473104492306442


## 改进

### Attention_CNN3(dropout1==0.1)

In [31]:
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Dense, Dropout, Input, BatchNormalization, GlobalMaxPooling1D

def attention_layer(inputs):
    attention_weights = Dense(units=128, activation='softmax')(inputs)
    attention_weights = Flatten()(attention_weights)
    attention_weights = Multiply()([inputs, attention_weights])
    return attention_weights

input_layer = Input(shape=(39, 1))
conv_layer1 = Conv1D(64, 3, padding="same", activation="relu")(input_layer)
pool_layer1 = MaxPooling1D(pool_size=(2))(conv_layer1)
conv_layer2 = Conv1D(128, 3, padding="same", activation="relu")(pool_layer1)
pool_layer2 = MaxPooling1D(pool_size=(2))(conv_layer2)
flatten_layer = Flatten()(pool_layer2)
dense_layer1 = Dense(128, activation="relu")(flatten_layer)
bn_layer1 = BatchNormalization()(dense_layer1)
dropout_layer1 = Dropout(0.01)(bn_layer1)
attention_layer1 = attention_layer(dropout_layer1)
dense_layer2 = Dense(64, activation="relu")(attention_layer1)
dropout_layer2 = Dropout(0.01)(dense_layer2)
output_layer = Dense(5, activation="softmax")(dropout_layer2)
attention_cnn3dropout1_model = Model(inputs=input_layer, outputs=output_layer)

attention_cnn3dropout1_model.compile(loss ='sparse_categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
print(attention_cnn3dropout1_model.summary())
start_time = time.time()
attention_cnn3dropout1_model.fit(X_train.reshape((-1,39,1)), Y_train.values.ravel(), epochs=10, batch_size=batch_size)
end_time = time.time()
attention_cnn3dropout1_time = end_time-start_time
print("Training time: ", attention_cnn3dropout1_time)

Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            [(None, 39, 1)]      0                                            
__________________________________________________________________________________________________
conv1d_8 (Conv1D)               (None, 39, 64)       256         input_5[0][0]                    
__________________________________________________________________________________________________
max_pooling1d_8 (MaxPooling1D)  (None, 19, 64)       0           conv1d_8[0][0]                   
__________________________________________________________________________________________________
conv1d_9 (Conv1D)               (None, 19, 128)      24704       max_pooling1d_8[0][0]            
____________________________________________________________________________________________

In [32]:
cnn3dropout1_preds_train = attention_cnn3dropout1_model.predict(X_train.reshape((-1,39,1)))
cnn3dropout1_test = attention_cnn3dropout1_model.predict(X_test.reshape((-1,39,1)))
print("attention_cnn3dropout1_model")
attention_cnn3dropout1_train_acc = accuracy_score(Y_train, np.argmax(cnn3dropout1_preds_train, axis=1))
attention_cnn3dropout1_test_acc = accuracy_score(Y_test, np.argmax(cnn3dropout1_test, axis=1))
attention_cnn3dropout1_train_f1 = f1_score(Y_train, np.argmax(cnn3dropout1_preds_train, axis=1), average='macro')
attention_cnn3dropout1_test_f1 = f1_score(Y_test, np.argmax(cnn3dropout1_test, axis=1), average='macro')
attention_cnn3dropout1_train_recall = recall_score(Y_train, np.argmax(cnn3dropout1_preds_train, axis=1), average='macro')
attention_cnn3dropout1_test_recall = recall_score(Y_test, np.argmax(cnn3dropout1_test, axis=1), average='macro')
print("Training Accuracy:", attention_cnn3dropout1_train_acc)
print("Testing Accuracy:", attention_cnn3dropout1_test_acc)
print("Training f1:", attention_cnn3dropout1_train_f1)
print("Testing f1:", attention_cnn3dropout1_test_f1)
print("Training recall:", attention_cnn3dropout1_train_recall)
print("Testing recall:", attention_cnn3dropout1_test_recall)

attention_cnn3dropout1_model
Training Accuracy: 0.9997620315739311
Testing Accuracy: 0.9997154315882072
Training f1: 0.7545396606558306
Testing f1: 0.7457175900173778
Training recall: 0.7334947908965008
Testing recall: 0.7243159901571472


### Attention_CNN3(dropout2==0.01)

In [33]:
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Dense, Dropout, Input, BatchNormalization, GlobalMaxPooling1D

def attention_layer(inputs):
    attention_weights = Dense(units=128, activation='softmax')(inputs)
    attention_weights = Flatten()(attention_weights)
    attention_weights = Multiply()([inputs, attention_weights])
    return attention_weights

input_layer = Input(shape=(39, 1))
conv_layer1 = Conv1D(64, 3, padding="same", activation="relu")(input_layer)
pool_layer1 = MaxPooling1D(pool_size=(2))(conv_layer1)
conv_layer2 = Conv1D(128, 3, padding="same", activation="relu")(pool_layer1)
pool_layer2 = MaxPooling1D(pool_size=(2))(conv_layer2)
flatten_layer = Flatten()(pool_layer2)
dense_layer1 = Dense(128, activation="relu")(flatten_layer)
bn_layer1 = BatchNormalization()(dense_layer1)
dropout_layer1 = Dropout(0.01)(bn_layer1)
attention_layer1 = attention_layer(dropout_layer1)
dense_layer2 = Dense(64, activation="relu")(attention_layer1)
dropout_layer2 = Dropout(0.01)(dense_layer2)
output_layer = Dense(5, activation="softmax")(dropout_layer2)
attention_cnn3dropout2_model = Model(inputs=input_layer, outputs=output_layer)

attention_cnn3dropout2_model.compile(loss ='sparse_categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
print(attention_cnn3dropout2_model.summary())
start_time = time.time()
attention_cnn3dropout2_model.fit(X_train.reshape((-1,39,1)), Y_train.values.ravel(), epochs=10, batch_size=batch_size)
end_time = time.time()
attention_cnn3dropout2_time = end_time-start_time
print("Training time: ", attention_cnn3dropout2_time)

Model: "model_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            [(None, 39, 1)]      0                                            
__________________________________________________________________________________________________
conv1d_10 (Conv1D)              (None, 39, 64)       256         input_6[0][0]                    
__________________________________________________________________________________________________
max_pooling1d_10 (MaxPooling1D) (None, 19, 64)       0           conv1d_10[0][0]                  
__________________________________________________________________________________________________
conv1d_11 (Conv1D)              (None, 19, 128)      24704       max_pooling1d_10[0][0]           
____________________________________________________________________________________________

In [34]:
cnn3dropout2_preds_train = attention_cnn3dropout2_model.predict(X_train.reshape((-1,39,1)))
cnn3dropout2_test = attention_cnn3dropout2_model.predict(X_test.reshape((-1,39,1)))
print("attention_cnn3dropout2_model")
attention_cnn3dropout2_train_acc = accuracy_score(Y_train, np.argmax(cnn3dropout2_preds_train, axis=1))
attention_cnn3dropout2_test_acc = accuracy_score(Y_test, np.argmax(cnn3dropout2_test, axis=1))
attention_cnn3dropout2_train_f1 = f1_score(Y_train, np.argmax(cnn3dropout2_preds_train, axis=1), average='macro')
attention_cnn3dropout2_test_f1 = f1_score(Y_test, np.argmax(cnn3dropout2_test, axis=1), average='macro')
attention_cnn3dropout2_train_recall = recall_score(Y_train, np.argmax(cnn3dropout2_preds_train, axis=1), average='macro')
attention_cnn3dropout2_test_recall = recall_score(Y_test, np.argmax(cnn3dropout2_test, axis=1), average='macro')
print("Training Accuracy:", attention_cnn3dropout2_train_acc)
print("Testing Accuracy:", attention_cnn3dropout2_test_acc)
print("Training f1:", attention_cnn3dropout2_train_f1)
print("Testing f1:", attention_cnn3dropout2_test_f1)
print("Training recall:", attention_cnn3dropout2_train_recall)
print("Testing recall:", attention_cnn3dropout2_test_recall)

attention_cnn3dropout2_model
Training Accuracy: 0.9997879308264482
Testing Accuracy: 0.9997432698024044
Training f1: 0.8847560348459765
Testing f1: 0.8636587401261986
Training recall: 0.8786176572957356
Testing recall: 0.8414356841924169


In [35]:
train_acc = {"attention_cnn2bn_train_acc":attention_cnn2bn_train_acc, 
            "attention_cnn3dropout1_train_acc":attention_cnn3dropout1_train_acc, "attention_cnn3dropout2_train_acc":attention_cnn3dropout2_train_acc}
test_acc = {"attention_cnn2bn_test_acc":attention_cnn2bn_test_acc,
            "attention_cnn3dropout1_test_acc":attention_cnn3dropout1_test_acc, "attention_cnn3dropout2_test_acc":attention_cnn3dropout2_test_acc}

In [36]:
train_f1 = {"attention_cnn2bn_train_f1":attention_cnn2bn_train_f1, 
            "attention_cnn3dropout1_train_f1":attention_cnn3dropout1_train_f1, "attention_cnn3dropout2_train_f1":attention_cnn3dropout2_train_f1}
test_f1 = {"attention_cnn2bn_test_f1":attention_cnn2bn_test_f1, 
            "attention_cnn3dropout1_test_f1":attention_cnn3dropout1_test_f1, "attention_cnn3dropout2_test_f1":attention_cnn3dropout2_test_f1}

In [37]:
train_recall = {"attention_cnn2bn_train_recall":attention_cnn2bn_train_recall, 
            "attention_cnn3dropout1_train_recall":attention_cnn3dropout1_train_recall, "attention_cnn3dropout2_train_recall":attention_cnn3dropout2_train_recall}
test_recall = {"attention_cnn2bn_test_recall":attention_cnn2bn_test_recall, 
            "attention_cnn3dropout1_test_recall":attention_cnn3dropout1_test_recall, "attention_cnn3dropout2_test_recall":attention_cnn3dropout2_test_recall}

In [38]:
print(train_acc, test_acc, train_f1, test_f1, train_recall, test_recall, sep='\n')

{'attention_cnn2bn_train_acc': 0.9997178504961078, 'attention_cnn3dropout1_train_acc': 0.9997620315739311, 'attention_cnn3dropout2_train_acc': 0.9997879308264482}
{'attention_cnn2bn_test_acc': 0.9996783139692778, 'attention_cnn3dropout1_test_acc': 0.9997154315882072, 'attention_cnn3dropout2_test_acc': 0.9997432698024044}
{'attention_cnn2bn_train_f1': 0.7548793939128742, 'attention_cnn3dropout1_train_f1': 0.7545396606558306, 'attention_cnn3dropout2_train_f1': 0.8847560348459765}
{'attention_cnn2bn_test_f1': 0.7543989859493948, 'attention_cnn3dropout1_test_f1': 0.7457175900173778, 'attention_cnn3dropout2_test_f1': 0.8636587401261986}
{'attention_cnn2bn_train_recall': 0.7485729355482988, 'attention_cnn3dropout1_train_recall': 0.7334947908965008, 'attention_cnn3dropout2_train_recall': 0.8786176572957356}
{'attention_cnn2bn_test_recall': 0.7473104492306442, 'attention_cnn3dropout1_test_recall': 0.7243159901571472, 'attention_cnn3dropout2_test_recall': 0.8414356841924169}


In [39]:
def dic_to_txt(dic, filepath):
    # 遍历字典的元素，将每项元素的key和value分拆组成字符串，注意添加分隔符和换行符
    file = open(filepath, 'w')
    for k, v in dic.items():
        file.write(str(k) + ' ' + str(v) + '\n')

    # 注意关闭文件
    file.close()
    print('done')

In [40]:
dic_to_txt(train_acc, './unprocess_AttentionCNN_reduce_train_acc.txt')
dic_to_txt(test_acc, './unprocess_AttentionCNN_reduce_test_acc.txt')

done
done


In [41]:
dic_to_txt(train_f1, './unprocess_AttentionCNN_reduce_train_f1.txt')
dic_to_txt(test_f1, './unprocess_AttentionCNN_reduce_test_f1.txt')

done
done


In [42]:
dic_to_txt(train_recall, './unprocess_AttentionCNN_reduce_train_recall.txt')
dic_to_txt(test_recall, './unprocess_AttentionCNN_reduce_test_recall.txt')

done
done


In [43]:
all_end_time = time.time()
total_tile = all_end_time - all_start_time
print("总共耗时：", total_tile)

总共耗时： 9651.361966609955
